## Parte 4: Renderização

In [1]:
def initOpenGL(dimensions):
    width, height = dimensions

    gl.glClearColor(0.0, 0.0, 0.0, 0.0)
    gl.glClearDepth(1.0)

    gl.glEnable(gl.GL_DEPTH_TEST)

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    
    K, _ = gera_matriz_parametros_intrinsecos()
    ajuste_intrinsecos_camera(width, height, K)

In [2]:
def resize_view(width, height, K):
    gl.glViewport(0, 0, width, height)

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    ajuste_intrinsecos_camera(width, height, K)

    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    glu.gluLookAt(0, 0, 5, 0, 0, 0, 0, 1, 0)

In [3]:
def background(img):
    id_textura = gl.glGenTextures(1)
    gl.glBindTexture(gl.GL_TEXTURE_2D, id_textura)

    fundo = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #fundo = cv2.flip(fundo, 0)
    
    gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MAG_FILTER, gl.GL_LINEAR)
    gl.glTexParameteri(gl.GL_TEXTURE_2D, gl.GL_TEXTURE_MIN_FILTER, gl.GL_LINEAR)
    gl.glTexImage2D(gl.GL_TEXTURE_2D, 0, gl.GL_RGB, 640, 480, 0, gl.GL_RGB, gl.GL_UNSIGNED_BYTE, fundo)

    gl.glDepthMask(gl.GL_FALSE)
    gl.glDisable( gl.GL_DEPTH_TEST )

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.glOrtho(0, 640, 480, 0.0, 0.0, 1)
    
    gl.glEnable( gl.GL_TEXTURE_2D )
    gl.glBegin(gl.GL_QUADS)
    gl.glTexCoord2f(0, 0); gl.glVertex2f(0, 0)
    gl.glTexCoord2f(1, 0); gl.glVertex2f(640, 0)
    gl.glTexCoord2f(1, 1); gl.glVertex2f(640, 480)
    gl.glTexCoord2f(0, 1); gl.glVertex2f(0, 480)
    gl.glEnd()

    gl.glBindTexture(gl.GL_TEXTURE_2D, 0)
    gl.glEnable( gl.GL_DEPTH_TEST )
    gl.glDepthMask(gl.GL_TRUE)

    gl.glFlush()
    

In [4]:
def desenha_pikachus_nas_posicoes(obj, infos_cena_atual):
    for posicao, orientacao in infos_cena_atual:   
        m = obtem_pose(posicao, orientacao)
        gl.glLoadMatrixf(m)
        gl.glCallList(obj.gl_list)

In [5]:
def cria_e_posiciona_cubo(m):

    vertices = (
        (1, -1, -1), (1, 1, -1), (-1, 1, -1),
        (-1, -1, -1),(1, -1, 1), (1, 1, 1),
        (-1, -1, 1), (-1, 1, 1)
    )

    arestas = (
        (2,1), (0,1), (0,3),
        (0,4), (2,3), (2,7),
        (6,3), (6,4), (6,7),
        (5,1), (5,4), (5,7)
    )
    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glLoadMatrixf(m)

    gl.glPushAttrib(gl.GL_CURRENT_BIT)
    gl.glLineWidth(4)

    gl.glBegin(gl.GL_LINES)
    gl.glColor3f(124/255, 252/255, 0)
    first_seg = True
    for aresta in arestas:
        for vertice in aresta:
            gl.glVertex3fv(vertices[vertice])
            if first_seg:
                first_seg = False
                gl.glColor3f(124/255, 252/255, 0)
            else:
                gl.glColor3f(230/255, 0., 126/255)
    gl.glEnd()

    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity()
    gl.glPopAttrib()
    gl.glFlush()

In [6]:
def desenha_cubos_nas_posicoes(infos_cena_atual):
    for posicao, orientacao in infos_cena_atual:   
        m = obtem_pose(posicao, orientacao)
        cria_e_posiciona_cubo(m)

In [7]:
def renderiza_pikachus(infos_cena_atual, modelo_objeto):    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glEnable(gl.GL_TEXTURE_2D)

    desenha_pikachus_nas_posicoes(modelo_objeto, infos_cena_atual)

In [8]:
def renderiza_cubos(infos_cena_atual):    
    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glEnable(gl.GL_TEXTURE_2D)

    desenha_cubos_nas_posicoes(infos_cena_atual)

In [9]:
def cria_cena(frames, frames_processados, modelo_pikachu):
    global frame_index
    global render_cubo
    global render_pikachu

    if frame_index >= len(frames_processados) and not render_pikachu:
        render_pikachu = True
        render_cubo = False
        frame_index = 0

    elif frame_index >= len(frames_processados) and render_pikachu:
        return

    gl.glMatrixMode(gl.GL_MODELVIEW)
    gl.glLoadIdentity()
    gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)

    infos_cena_atual = frames_processados[frame_index]

    background(frames[frame_index])
    gl.glMatrixMode(gl.GL_PROJECTION)
    gl.glLoadIdentity() 
    K, _ = gera_matriz_parametros_intrinsecos()
    ajuste_intrinsecos_camera(640, 480, K)

    if render_pikachu:
        renderiza_pikachus(infos_cena_atual, modelo_pikachu)
    else:
        renderiza_cubos(infos_cena_atual)

    glut.glutSwapBuffers()

    time.sleep(1/18) # Valor qualquer pro vídeo nao ficar muito rapido
    frame_index = frame_index + 1

In [10]:
def idleCallback():
    glut.glutPostRedisplay()

Lógica para renderização do Pikachu:

In [11]:
def realidade_aumentada_pikachu(frames, frames_processados):
    dimensoes_frame = (640, 480)

    K, _ = gera_matriz_parametros_intrinsecos()

    glut.glutInit()
    glut.glutInitDisplayMode(glut.GLUT_RGBA | glut.GLUT_DOUBLE)
    glut.glutSetOption(glut.GLUT_ACTION_ON_WINDOW_CLOSE, glut.GLUT_ACTION_CONTINUE_EXECUTION)
    glut.glutInitWindowSize(*dimensoes_frame)
    janela = glut.glutCreateWindow(b'Realidade Aumentada - Pikachu')
    
    initOpenGL(dimensoes_frame)

    global frame_index
    frame_index = 0

    global render_pikachu
    render_pikachu = False

    global render_cubo
    render_cubo = True

    obj = OBJ("Pikachu.obj", swapyz=True)
    
    display_callback = lambda: cria_cena(frames, frames_processados, obj)
    reshape_callback = lambda w, h: resize_view(w, h, K)

    glut.glutDisplayFunc(display_callback)
    glut.glutReshapeFunc(reshape_callback)
    glut.glutIdleFunc(idleCallback)
    glut.glutMainLoop()

In [12]:
alvo, cantos_alvo = obtem_alvo()
frames = captura_frames()
mapa_rotacao, mapa_coordenada = gera_mapa_rotacoes_alvo(alvo)
frames_processados = processa_frames(frames, alvo, mapa_rotacao)

NameError: name 'obtem_alvo' is not defined

In [13]:
realidade_aumentada_pikachu(frames, frames_processados)

NameError: name 'frames' is not defined

In [25]:
alvo, cantos_alvo = obtem_alvo()
frames = captura_frames()
mapa_rotacao, mapa_coordenada = gera_mapa_rotacoes_alvo(alvo)
frames_processados = processa_frames(frames, alvo, mapa_rotacao)